In [ ]:
import numpy as np
from skimage import io, color, filters, measure
import scipy.ndimage as ndi
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
import matplotlib.pyplot as plt
from extract_circular_points import extract_circular_points

# Function to perform image segmentation
def perform_segmentation(image_path):
    # Load the image and convert it to grayscale
    img = io.imread(image_path)
    img_gray = color.rgb2gray(img)

    # Compute the distance transform
    distance = ndi.distance_transform_edt(img_gray)

    # Calculate the threshold using Otsu's method
    threshold = filters.threshold_otsu(img_gray)

    # Find peaks in the distance map
    markers = peak_local_max(distance, min_distance=10, labels=img_gray > threshold)

    # Resize markers to the same shape as the mask
    markers = np.resize(markers, img_gray.shape)

    # Perform watershed segmentation
    labeled_image = watershed(-distance, markers, mask=img_gray)

    return labeled_image

# Function to extract features from segmented regions
def extract_features_from_segmentation(labeled_image, img_gray, label, filepath):
    # Calculate the features from each segmented region
    regions = measure.regionprops(labeled_image, intensity_image=img_gray)
    features = []
    concave_points_values = []
    symmetry_values = []
    fractal_dimension_values = []

    count = 0
    for i, region in enumerate(regions):
        # print("region", region, region.perimeter, region.area, region.convex_area, region.convex_image)
        if region.area == 0 or region.perimeter == 0:
            continue
        radius = region.perimeter / (2.0 * np.pi)
        texture = np.std(img_gray[region.slice])
        perimeter = region.perimeter
        area = region.area
        smoothness = (perimeter ** 2) / area
        compactness = (perimeter ** 2) / area
        concavity = region.convex_area - area
        concave_points = len(region.convex_image) - np.sum(region.convex_image)
        IMG_SIZE = max(img_gray.shape)
        symmetry = np.sum(np.abs(img_gray[region.slice] - np.flipud(img_gray[region.slice]))) / (IMG_SIZE ** 2)
        fractal_dimension = np.log(4 * np.pi * area) / (2 * np.log(perimeter))

        if concave_points != 0:
            concave_points_values.append(concave_points)
        if concave_points == 0:
            concave_points = np.mean(concave_points_values)

        if symmetry != 0:
            symmetry_values.append(symmetry)
        if symmetry == 0:
            symmetry = np.mean(symmetry_values)

        if fractal_dimension != 0:
            fractal_dimension_values.append(fractal_dimension)
        if fractal_dimension == 0:
            fractal_dimension = np.mean(fractal_dimension_values)

        count = count+1
        feature = {
            'radius': radius,
            'texture': texture,
            'perimeter': perimeter,
            'area': area,
            'smoothness': smoothness,
            'compactness': compactness,
            'concavity': concavity,
            'concave_points': concave_points,
            'symmetry': symmetry,
            'fractal_dimension': fractal_dimension,
            'label': label,
            'path': filepath,
            'path_id': count
        }
        print("feature", feature)
        # if(i==0):
        #     print("labeled_image", labeled_image, label, feature)
        #     plt.imshow(labeled_image)
        #     plt.axis('off')  # Turn off the axis labels
        #     plt.show()
        features.append(feature)
    return features

